In [ ]:
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint


# Load The Model

In [ ]:
HEIGHT = 300
WIDTH = 300

base_model = ResNet50(weights='imagenet',include_top=False, input_shape=(HEIGHT, WIDTH, 3))

# Load Training Data

In [ ]:
TRAIN_DIR = 'output/train'
HEIGHT = 300
WIDTH = 300
BATCH_SIZE = 16

train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
)

train_generator = train_datagen.flow_from_directory(TRAIN_DIR, 
                                                    target_size=(HEIGHT, WIDTH),
                                                    batch_size=BATCH_SIZE)

# Load Validation Data

In [ ]:
VAL_DIR = 'output/val'
HEIGHT = 300
WIDTH = 300
BATCH_SIZE = 16

val_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
)

val_generator = val_datagen.flow_from_directory(VAL_DIR, 
                                                    target_size=(HEIGHT, WIDTH),
                                                    batch_size=BATCH_SIZE)

# Build the model

In [ ]:
def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

class_list = ["ana","close","continue","eat","fen","listen","open","short",
              "tall","watch"]
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS,
                                      num_classes=len(class_list))

# Training 

In [ ]:
NUM_EPOCHS = 10
BATCH_SIZE = 16
num_train_images = 15039
num_val_images = 2816 

adam = Adam(lr=0.00001)
model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

filepath="/content/drive/My Drive/" + "New_ResNet50_model_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
callbacks_list = [checkpoint]

history = model.fit_generator(generator=train_generator, epochs=NUM_EPOCHS, workers=8, 
                                       steps_per_epoch=num_train_images // BATCH_SIZE,
                                       validation_data=val_generator, 
                                       validation_steps=num_val_images // BATCH_SIZE,
                                       shuffle=True, callbacks=callbacks_list)
